# AG Grid Enterprise with anywidget

This notebook demonstrates using AG Grid Enterprise features with the anywidget backend.

**Note:** Enterprise features require a valid license key from AG Grid. Without a valid license, you will see a watermark and console warnings.

In [ ]:
import numpy as np
import pandas as pd

from nbappinator import ColMd, create_grid

In [ ]:
# Kitchen sink sample data - more rows for grouping demo
np.random.seed(42)

sectors = [
    "Technology",
    "Technology",
    "Technology",
    "Technology",
    "Technology",
    "Finance",
    "Finance",
    "Finance",
    "Finance",
    "Healthcare",
    "Healthcare",
    "Healthcare",
    "Consumer",
    "Consumer",
    "Consumer",
    "Consumer",
    "Energy",
    "Energy",
    "Energy",
]

regions = [
    "North America",
    "North America",
    "Europe",
    "Asia",
    "North America",
    "North America",
    "Europe",
    "Asia",
    "North America",
    "North America",
    "Europe",
    "Asia",
    "North America",
    "Europe",
    "Asia",
    "North America",
    "North America",
    "Europe",
    "Asia",
]

df = pd.DataFrame(
    {
        "company": [
            "Apple",
            "Microsoft",
            "SAP",
            "Sony",
            "NVIDIA",
            "JPMorgan",
            "HSBC",
            "Mitsubishi UFJ",
            "Goldman Sachs",
            "Johnson & Johnson",
            "Roche",
            "Takeda",
            "Amazon",
            "LVMH",
            "Alibaba",
            "Walmart",
            "ExxonMobil",
            "Shell",
            "PetroChina",
        ],
        "sector": sectors,
        "region": regions,
        "price": np.random.uniform(50, 500, 19).round(2),
        "change_pct": np.random.uniform(-0.08, 0.08, 19),
        "volume": np.random.randint(5_000_000, 80_000_000, 19),
        "market_cap": np.random.uniform(0.1e12, 3e12, 19),
        "pe_ratio": np.random.uniform(10, 40, 19).round(1),
        "dividend_yield": np.random.uniform(0, 0.05, 19),
    }
)

## Kitchen Sink Enterprise Example

This example demonstrates multiple enterprise features:
- **Row Grouping**: Drag columns to the group panel to group data
- **Range Selection**: Click and drag to select cell ranges (Ctrl+C to copy)
- **Clipboard**: Copy/paste support for selected ranges
- **Excel Export**: Right-click context menu includes Excel export
- **Aggregation**: Grouped rows show sum/avg of numeric columns

In [ ]:
# Dummy license key - replace with your actual AG Grid Enterprise license
DUMMY_LICENSE = "your-company-name_your-license-key-here_Evaluation_1234567890"

columns = [
    ColMd(name="company", label="Company", pinned=True),
    ColMd(name="sector", label="Sector"),
    ColMd(name="region", label="Region"),
    ColMd(name="price", label="Price", format="dec", precision=2),
    ColMd(name="change_pct", label="Change %", format="perc", precision=2),
    ColMd(name="volume", label="Volume", format="mag_si", precision=1),
    ColMd(name="market_cap", label="Market Cap", format="mag_si", precision=2),
    ColMd(name="pe_ratio", label="P/E Ratio", format="dec", precision=1),
    ColMd(name="dividend_yield", label="Div Yield", format="perc", precision=2),
]

# Kitchen sink enterprise grid with all features
grid_enterprise = create_grid(
    df,
    col_md=columns,
    height=500,
    enterprise=True,
    license_key=DUMMY_LICENSE,
    grid_options={
        # Row Grouping - drag sector/region to group panel
        "rowGroupPanelShow": "always",
        "groupDisplayType": "multipleColumns",
        "groupDefaultExpanded": 1,
        # Make sector and region groupable by default
        "columnDefs": [
            {"field": "sector", "rowGroup": True, "hide": True, "enableRowGroup": True},
            {"field": "region", "enableRowGroup": True},
        ],
        # Aggregation for grouped rows
        "aggFuncs": {
            "sum": "sum",
            "avg": "avg",
        },
        "groupIncludeFooter": True,
        "groupIncludeTotalFooter": True,
        # Range Selection (for copy/paste)
        "enableRangeSelection": True,
        "enableRangeHandle": True,
        # Clipboard
        "enableClipboard": True,
        "copyHeadersToClipboard": True,
        # Excel Export (right-click context menu)
        "defaultExcelExportParams": {
            "fileName": "grid_export.xlsx",
            "sheetName": "Data",
        },
        # Context menu with export options
        "getContextMenuItems": None,  # Uses default which includes export
        # Status bar showing aggregations
        "statusBar": {
            "statusPanels": [
                {"statusPanel": "agTotalAndFilteredRowCountComponent"},
                {"statusPanel": "agSelectedRowCountComponent"},
                {"statusPanel": "agAggregationComponent"},
            ]
        },
        # Side bar with columns/filters panels
        "sideBar": {
            "toolPanels": ["columns", "filters"],
            "defaultToolPanel": "",
        },
    },
)
grid_enterprise

## App Example with Cell Click Callback

In [ ]:
import nbappinator as nb

# Store app reference for callback
_app = None


def on_cell_click(event):
    """Callback when a cell is clicked"""
    global _app
    if _app is None:
        return
    with _app.messages:
        col = event.get("col_clicked")
        value = event.get("value_clicked")
        row_data = event.get("data", {})
        company = row_data.get("company", "Unknown")
        print(f"Clicked: {company} -> {col} = {value}")


def update_grid(app):
    global _app
    _app = app
    with app.messages:
        app.status("Loading grid...")

        columns = [
            ColMd(name="company", label="Company", pinned=True),
            ColMd(name="sector", label="Sector"),
            ColMd(name="region", label="Region"),
            ColMd(name="price", label="Price", format="dec", precision=2),
            ColMd(name="market_cap", label="Market Cap", format="mag_si", precision=2),
        ]

        app.tab(0).clear().dataframe(
            "grid",
            df,
            columns=columns,
            on_click=on_cell_click,
            height=350,
            enterprise=True,
            license_key=DUMMY_LICENSE,
            grid_options={
                "rowGroupPanelShow": "always",
                "enableRangeSelection": True,
            },
        )

        app.done("Click any cell to see its value printed below")


app = nb.App(tabs=["Enterprise Grid"], footer="Messages", header="Controls")
app.config.button("refresh", on_click=update_grid, label="Refresh Grid", status=True)
update_grid(app)
app.display()

In [ ]:
# Hierarchical organization data
tree_df = pd.DataFrame(
    {
        "path": [
            "Global",
            "Global/North America",
            "Global/North America/Technology",
            "Global/North America/Technology/Apple",
            "Global/North America/Technology/Microsoft",
            "Global/North America/Technology/NVIDIA",
            "Global/North America/Finance",
            "Global/North America/Finance/JPMorgan",
            "Global/North America/Finance/Goldman Sachs",
            "Global/Europe",
            "Global/Europe/Technology",
            "Global/Europe/Technology/SAP",
            "Global/Europe/Finance",
            "Global/Europe/Finance/HSBC",
            "Global/Asia",
            "Global/Asia/Technology",
            "Global/Asia/Technology/Sony",
            "Global/Asia/Finance",
            "Global/Asia/Finance/Mitsubishi UFJ",
        ],
        "revenue": [
            5000e9,
            2500e9,
            1500e9,
            400e9,
            200e9,
            60e9,
            800e9,
            150e9,
            50e9,
            1500e9,
            500e9,
            30e9,
            400e9,
            50e9,
            1000e9,
            300e9,
            80e9,
            200e9,
            40e9,
        ],
        "employees": [
            2_000_000,
            1_000_000,
            500_000,
            160_000,
            220_000,
            30_000,
            400_000,
            290_000,
            45_000,
            600_000,
            200_000,
            110_000,
            300_000,
            220_000,
            400_000,
            150_000,
            110_000,
            100_000,
            80_000,
        ],
        "growth_pct": [
            0.08,
            0.10,
            0.15,
            0.12,
            0.18,
            0.45,
            0.05,
            0.03,
            0.08,
            0.06,
            0.12,
            0.08,
            0.04,
            0.02,
            0.09,
            0.20,
            0.15,
            0.06,
            0.04,
        ],
    }
)

tree_columns = [
    ColMd(name="revenue", label="Revenue", format="mag_si", precision=1),
    ColMd(name="employees", label="Employees", format="mag_si", precision=0),
    ColMd(name="growth_pct", label="Growth", format="perc", precision=1),
]

# Tree grid - requires enterprise!
tree_grid = create_grid(
    tree_df,
    is_tree=True,
    pathcol="path",
    pathdelim="/",
    col_md=tree_columns,
    height=450,
    enterprise=True,
    license_key=DUMMY_LICENSE,
    grid_options={
        "groupDefaultExpanded": 2,  # Expand first 2 levels
        "enableRangeSelection": True,
    },
)
tree_grid